In [15]:
from pathlib import Path
import chromadb
from chromadb.utils import embedding_functions

# Path
Relative_Database_path = "./chroma_Data_v_final"
Absolute_Database_path = Path(Relative_Database_path).resolve()
collection_name = "anlp_rag_collection"

# Initialize Chroma
client = chromadb.PersistentClient(path=str(Absolute_Database_path))
print(f"[INFO] ChromaDB client initialized at: {Absolute_Database_path}")

# Correct embedding function: use model_name (primitive), not a model instance
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# Load existing collection
collection = client.get_collection(
    name=collection_name,
    embedding_function=embedding_function
)

print(f"[SUCCESS] Loaded collection '{collection_name}'")
print(f"[INFO] Count: {collection.count()}")


[INFO] ChromaDB client initialized at: C:\Users\micro\Desktop\Abhinav college\Resources\Sem 7\Advanced NLP\Assignment 2\RAG-A2\VectorDB\chroma_Data_v_final
[SUCCESS] Loaded collection 'anlp_rag_collection'
[INFO] Count: 126


5 Groq API calls => trying to handle rate limits

In [16]:
# === Groq + RAG + RAGAS Evaluation (Fully Patched Version) ===
# Prereqs:
# pip install ragas datasets groq tqdm sentence-transformers numpy

import os
import json
import time
import numpy as np
from datetime import datetime
from tqdm import tqdm
from collections import deque
from datasets import Dataset

from groq import Groq
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy
from ragas.embeddings.base import HuggingfaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_core.prompt_values import PromptValue
from langchain_core.outputs import Generation, LLMResult
from ragas.llms.base import BaseRagasLLM as RagasBaseLLM
from ragas.run_config import RunConfig

# ============================================================
# NEW: LOG FILE
# ============================================================
GROQ_LOG_PATH = "../RAG Results/groq_logs.txt"

def log_groq_call(model, prompt, response):
    """Append Groq prompt/response to log file."""
    with open(GROQ_LOG_PATH, "a", encoding="utf-8") as f:
        f.write("\n===============================================\n")
        f.write(f"TIMESTAMP: {datetime.now()}\n")
        f.write(f"MODEL: {model}\n\n")
        f.write("PROMPT:\n")
        f.write(prompt + "\n\n")
        f.write("RESPONSE:\n")
        f.write(response + "\n")
        f.write("===============================================\n\n")


# ============================================================
# 🔥 5 GROQ API KEYS - ROUND ROBIN
# ============================================================
API_KEYS = [
    "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",
    "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",
    "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",
    "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",
    "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",
]

rr_index = 0
def get_next_client():
    global rr_index
    client = Groq(api_key=API_KEYS[rr_index])
    rr_index = (rr_index + 1) % len(API_KEYS)
    return client

# ============================================================
# GLOBAL SAFE RATE LIMITER
# ============================================================
MAX_RPM = 25
WINDOW = 60
global_request_times = deque()

def wait_for_slot():
    now = time.time()

    while global_request_times and now - global_request_times[0] > WINDOW:
        global_request_times.popleft()

    if len(global_request_times) >= MAX_RPM:
        sleep_time = WINDOW - (now - global_request_times[0]) + 0.5
        print(f"[GLOBAL LIMIT] Waiting {sleep_time:.1f}s...")
        time.sleep(sleep_time)

    global_request_times.append(time.time())


# ============================================================
# CONFIG
# ============================================================
testbed_path = "../RAG Results/test_bed.json"
output_metrics_path = "../RAG Results/multiquery_rag_metrics.txt"
cached_answers_path = "../RAG Results/cached_rag_answers.json"

TOP_K = 3

GROQ_RAG_MODEL = "llama-3.3-70b-versatile"
GROQ_RAGAS_MODEL = "llama-3.1-8b-instant"
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

REQUEST_DELAY = 12.0
MAX_RETRIES = 5


# ============================================================
# Load testbed
# ============================================================
print("Exists:", os.path.exists(testbed_path))
print("Size:", os.path.getsize(testbed_path), "bytes")

with open(testbed_path, "r") as f:
    test_data = json.load(f)

print(f"[INFO] Loaded {len(test_data)} QA pairs.")


# ============================================================
# Groq generation helper
# ============================================================
def generate_with_groq(prompt, model_name=GROQ_RAG_MODEL):
    for attempt in range(MAX_RETRIES):
        try:
            wait_for_slot()
            client = get_next_client()

            response = client.chat.completions.create(
                model=model_name,
                temperature=0.7,
                messages=[{
                    "role": "user",
                    "content": prompt + "\n\nIMPORTANT: Respond in plain text. No lists. No JSON."
                }]
            )
            text = response.choices[0].message.content.strip()

            log_groq_call(model_name, prompt, text)
            time.sleep(REQUEST_DELAY)

            return text

        except Exception as e:
            if "rate_limit" in str(e).lower():
                sleep_time = REQUEST_DELAY * (attempt + 2)
                print(f"[WARN] Rate limit. Sleeping {sleep_time}s")
                time.sleep(sleep_time)
            else:
                print(f"[ERROR] {e}")
                if attempt == MAX_RETRIES - 1:
                    return None


# ============================================================
# Groq wrapper for RAGAS scoring (JSON FORCING FIX APPLIED)
# ============================================================
class GroqRagasLLM(RagasBaseLLM):
    def __init__(self, model_name):
        super().__init__(run_config=RunConfig())
        self.model_name = model_name

    def _extract_text(self, prompt: PromptValue):
        if hasattr(prompt, "to_string"):
            return prompt.to_string()
        return str(prompt)

    def generate_text(self, prompt: PromptValue, n=1, temperature=0.0, stop=None, callbacks=None):
        raw_prompt = self._extract_text(prompt)

        # STRICT JSON WRAPPER — FIXES ALL PARSER ERRORS
        json_prompt = f"""
You MUST respond ONLY with valid JSON.
No explanations. No natural language. No commentary. No markdown.
If an array is expected, return an array.
If a dictionary is expected, return a dictionary.
Your entire response MUST be valid JSON.

Task:
{raw_prompt}
"""

        generations = []

        for _ in range(n):
            for attempt in range(MAX_RETRIES):
                try:
                    wait_for_slot()
                    client = get_next_client()

                    response = client.chat.completions.create(
                        model=self.model_name,
                        temperature=temperature,
                        messages=[{"role": "user", "content": json_prompt}]
                    )

                    text = response.choices[0].message.content.strip()
                    log_groq_call(self.model_name, json_prompt, text)

                    generations.append([Generation(text=text)])
                    break

                except Exception as e:
                    if "rate_limit" in str(e).lower():
                        time.sleep(REQUEST_DELAY)
                    else:
                        if attempt == MAX_RETRIES - 1:
                            generations.append([Generation(text='{"error": "LLM failed"}')])
                        else:
                            time.sleep(REQUEST_DELAY)

        return LLMResult(generations=generations)

    async def agenerate_text(self, *args, **kwargs):
        return self.generate_text(*args, **kwargs)

    def is_finished(self, response):
        return True


# ============================================================
# Generate RAG answers (with caching)
# ============================================================
records = []

if os.path.exists(cached_answers_path):
    print("[INFO] Loading cached answers...")
    with open(cached_answers_path, "r") as f:
        records = json.load(f)

if not records:
    print("[INFO] Generating new RAG answers...")
    for item in tqdm(test_data):
        q = item["question"]
        gt = item["ideal_answer"]

        retrieved = collection.query(query_texts=[q], n_results=TOP_K)
        docs = retrieved["documents"][0]
        context = "\n".join(docs)

        prompt = f"Context:\n{context}\n\nQuestion:\n{q}\n\nAnswer:"
        ans = generate_with_groq(prompt)
        if not ans:
            ans = "[Error: failed generation]"

        records.append({
            "question": q,
            "contexts": docs,
            "answer": ans,
            "ground_truth": gt
        })

    with open(cached_answers_path, "w") as f:
        json.dump(records, f, indent=2)


# ============================================================
# Convert to HF dataset
# ============================================================
dataset = Dataset.from_list(records)


# ============================================================
# Embedding wrapper (ASYNC METHODS FIXED)
# ============================================================
class CustomHuggingfaceEmbeddings(HuggingfaceEmbeddings):
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return self.model.encode(texts, show_progress_bar=False).tolist()

    def embed_query(self, text):
        return self.model.encode([text], show_progress_bar=False)[0].tolist()

    async def aembed_documents(self, texts):
        return self.embed_documents(texts)

    async def aembed_query(self, text):
        return self.embed_query(text)

embeddings = CustomHuggingfaceEmbeddings(EMBED_MODEL)


# ============================================================
# Run RAGAS evaluation
# ============================================================
llm = GroqRagasLLM(GROQ_RAGAS_MODEL)

print("[INFO] Running RAGAS evaluation...")
start = time.time()

results = evaluate(
    dataset=dataset,
    metrics=[faithfulness, answer_relevancy],
    llm=llm,
    embeddings=embeddings
)

end = time.time()
elapsed = (end - start) / 60

faith_avg = float(np.mean(results["faithfulness"]))
rel_avg = float(np.mean(results["answer_relevancy"]))

with open(output_metrics_path, "w") as f:
    f.write("=== RAG Evaluation Metrics ===\n")
    f.write(f"Time: {datetime.now()}\n")
    f.write(f"Duration: {elapsed:.2f} minutes\n")
    f.write(f"Faithfulness: {faith_avg:.4f}\n")
    f.write(f"Answer Relevancy: {rel_avg:.4f}\n")
    f.write("Full Results:\n")
    f.write(str(results))

print("DONE!")
print("Faithfulness:", faith_avg)
print("Answer Relevancy:", rel_avg)


Exists: True
Size: 7215 bytes
[INFO] Loaded 37 QA pairs.
[INFO] Generating new RAG answers...


  0%|          | 0/37 [00:00<?, ?it/s]

100%|██████████| 37/37 [08:18<00:00, 13.48s/it]


[INFO] Running RAGAS evaluation...


Evaluating:   0%|          | 0/74 [00:00<?, ?it/s]

[GLOBAL LIMIT] Waiting 9.1s...
[GLOBAL LIMIT] Waiting 0.8s...
[GLOBAL LIMIT] Waiting 0.5s...
[GLOBAL LIMIT] Waiting 0.6s...
[GLOBAL LIMIT] Waiting 1.0s...
[GLOBAL LIMIT] Waiting 1.1s...
[GLOBAL LIMIT] Waiting 8.7s...


Evaluating:  24%|██▍       | 18/74 [03:20<07:54,  8.47s/it]  

[GLOBAL LIMIT] Waiting 8.0s...


Evaluating: 100%|██████████| 74/74 [08:55<00:00,  7.23s/it]


DONE!
Faithfulness: 0.7333547833547833
Answer Relevancy: 0.7745587596295099


In [17]:
with open("../RAG Results/cached_rag_answers.json", "w") as f:
    json.dump(records, f, indent=2)
